In [40]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy.fft import fftn, ifftn
import imageio

def k_vector(npoints):
    k1 = np.arange(npoints/2+1)
    k2 = np.arange(-npoints/2+1, 0)
    
    kvector = 2*np.pi/ npoints* np.concatenate([k1, k2])
    return kvector

def pk_vector_delta(kvector, k0, dk0):
    
    npoints = len(kvector)
    shape = [npoints] * dim
    kk = np.zeros(shape)
    
      
    for i, j, k in itertools.product(range(npoints), range(npoints), range(npoints)):
        kk[i, j, k] = np.sqrt(kvector[i]**2 + kvector[j]**2 + kvector[k]**2)
               
    print("min_kk:",np.min(kk), "max_kk:", np.max(kk))
    
    pk=np.zeros_like(kk)
    sel=(kk > k0-dk0/2)*(kk < k0+dk0/2)
    pk[sel]=1
    
    xx, yy, zz = np.mgrid[0:npoints, 0:npoints, 0:npoints]
    r = np.sqrt((xx-npoints/2)**2 + (yy-npoints/2)**2 + (zz-npoints/2)**2)

    mask = r > npoints/2  
    mask2 = r < 0.8 * npoints/2
    pk[mask*mask2]=0
    #pk[mask]=0

    #pk[0,0,0] = 0 #changed to 1 from 0 (on aug 12, 2023)
    #print("pk:", pk,'\n')
    return pk


def field_delta(k0, dk0, npoints):
    
    k = k_vector(npoints)
    pk = pk_vector_delta(k, k0, dk0)
    Pk1 = np.zeros_like(pk)
    #Pk1 /= Pk1.sum()
    Pk1 = pk

    field=np.random.randn(npoints, npoints, npoints)
    fft_field=fftn(field)
    
    pspect_field = np.sqrt(Pk1) * fft_field
    new_field = np.real(ifftn(pspect_field))
    
    return new_field

#    def pk_vector_powerlaw(kvector, n):
#    
#    npoints = len(kvector)
#    shape = [npoints] * dim
#    kk = np.zeros(shape)
#    
#      
#    for i, j, k in itertools.product(range(npoints), range(npoints), range(npoints)):
#        kk[i, j, k] = np.sqrt(kvector[i]**2 + kvector[j]**2 + kvector[k]**2)
#               
#    print(np.min(kk),np.max(kk))
#    
#    pk=1/np.abs(kk)**n
#    
#    xx, yy, zz = np.mgrid[0:npoints, 0:npoints, 0:npoints]
#    r = np.sqrt((xx-npoints/2)**2 + (yy-npoints/2)**2 + (zz-npoints/2)**2)
#
#    mask = r > npoints/2
#    mask2 = r < 0.8 * npoints/2
#    pk[mask*mask2]=0
#    #pk[mask]=0
#
#    pk[0,0,0] = 0
#    
#    return pk


#def field_powerlaw(n, npoints):
#    
#    k = k_vector(npoints)
#    pk = pk_vector_powerlaw(k, n)
#    Pk1 = np.zeros_like(pk)
#    #Pk1 /= Pk1.sum()
#    Pk1 = pk
#
#    field=np.random.randn(npoints, npoints, npoints)
#    fft_field=fftn(field)
#    
#    pspect_field = np.sqrt(Pk1) * fft_field
#    new_field = np.real(ifftn(pspect_field))
#    
#   return new_field

if __name__=='__main__':

    npoints = 100
    dim = 3
    n = 3
    
    #new_field=field_powerlaw(n, npoints)
    new_field=field_delta(0.5, 0.05, npoints)
    
    A=0.1 # fluctuation amplitude standard deviation
    norm_field=new_field/np.std(new_field)*A
    
    print("mean_norm_field:", np.mean(norm_field), "std_norm_field:", np.std(norm_field), np.max(norm_field))
    plt.hist(norm_field.flatten())
    plt.show()
    
    xx, yy, zz = np.mgrid[0:npoints, 0:npoints, 0:npoints]
    r = np.sqrt((xx-npoints/2)**2 + (yy-npoints/2)**2 + (zz-npoints/2)**2)
    
    with imageio.get_writer('./new_field_test_5.gif', mode='I') as writer:
        for slice in new_field:
            writer.append_data(slice)    
    
    with imageio.get_writer('./norm_field_5.gif', mode='I') as writer:
        for slice in norm_field:
            writer.append_data(slice)

#fig=plt.figure()
#ax=fig.add_subplot(111, projection='3d')
#ax.voxels(new_field)
#plt.show()


min_kk: 0.0 max_kk: 5.441398092702654
mean_norm_field: -9.237055564881302e-20 std_norm_field: 0.09999999999999999 0.41739239673210327


In [ ]:
#Below this is an earlier version of the same code with pk vector in form of power law. 

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import itertools
import imageio

from scipy.fft import ifftn, fftn

#creating k vector
def create_kvector(npoints):
    
    if npoints%2 == 1:
        raise ValueError('Please use even number of points')    
    
    vector1 = np.arange(npoints/2+1)
    vector2 = np.arange(-npoints/2+1, 0)
    kvector = npoints*(2*np.pi/ npoints * np.concatenate([vector1, vector2]))
    
    return kvector


In [33]:
#creating pk vector
def create_Pk(kvector, n, ndim):
    
    #if ndim not in [2, 3]:
    #    raise ValueError(f'{ndim} gaussian field not suppported.')
    
    npoints = len(kvector)
    shape = [npoints] * ndim
    kk = np.zeros(shape)
    
    #if ndim == 2:
    #    for i, j in itertools.product(range(npoints), range(npoints)):
    #        kk[i, j] = np.sqrt(kvector[i]**2 + kvector[j]**2)
    #elif ndim == 3:
    for i, j, k in itertools.product(range(npoints), range(npoints), range(npoints)):
        kk[i, j, k] = np.sqrt(kvector[i]**2 + kvector[j]**2 + kvector[k]**2) 
        #kk[0:50, 0:50, 0:50]=0 
            
    Pk = 1/np.abs(kk**n)
    #if ndim == 2:
    #    Pk[0, 0] = 0
    #elif ndim == 3:
    Pk[0, 0, 0] = 0
    
    return Pk


In [34]:
#int__name__='__main__'
# initialize variables
npoints = 30
n = 4
ndim = 3

# generate random field
field = np.random.randn(npoints, npoints, npoints)
fft_field = fftn(field)
print(field,np.average(field))
# generate kvector
k = create_kvector(npoints)
Pk = create_Pk(k, n, ndim)
Pk1 = np.zeros_like(Pk)
#Pk1[50:, 50:, 50:] = 0
#Pk1 /= Pk1.sum()
Pk1 = Pk

Pspec_fft = np.sqrt(Pk1) * fft_field
new_field = np.real(ifftn(Pspec_fft))

xx, yy, zz = np.mgrid[:npoints, :npoints, :npoints]


r = np.sqrt((xx-npoints/2)**2 + (yy-npoints/2)**2 + (zz-npoints/2)**2)

mask = r > npoints/2
#mask2 = r < npoints/10

new_field[mask] = 0
#new_field[mask2] = np.nan

with imageio.get_writer('./new_field1.gif', mode='I') as writer:
    for slice in new_field:
        writer.append_data(slice)
        
with imageio.get_writer('./initial_field1.gif', mode='I') as writer:
    for slice in field:
        writer.append_data(slice) 

    
# generate a 3D cube with random values
#cube = new_field

## create a bubble by setting the values within a sphere to 0
##x, y, z = np.ogrid[0:100, 0:100, 0:100]
#bubble = (xx - 50)**2 + (yy - 50)**2 + (zz - 50)**2 > 50**2
#cube[bubble] = 0
#
# plot the cube
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.voxels(cube, edgecolor='k')
#plt.savefig('cube with bubble.png')
#plt.show()



[[[ 1.28763429 -0.26421241 -1.89959629 ...  1.50585753  0.86935438
   -1.09372431]
  [-0.90942476  0.00519017  1.22194101 ... -0.23604904  0.91511958
    1.48011805]
  [-1.13928353 -0.4420516  -1.07842499 ...  0.06223239 -1.08673608
    0.3385157 ]
  ...
  [-0.66337214  0.66442748  1.13073512 ... -0.30738021 -0.17382424
   -0.42837527]
  [ 1.08621748 -0.51234058 -1.15322563 ...  0.95956935 -0.23766274
   -1.68944694]
  [-0.21615318 -0.30974703  0.45203683 ...  1.01073004  0.22287363
    0.74103208]]

 [[-0.14745109  0.54241571 -0.2328126  ...  1.45697979  0.55666911
   -0.12657231]
  [-0.68845576  1.97997595  1.15790824 ...  0.25714378 -0.32479462
   -0.88507926]
  [-0.70788156  0.27068736 -1.20061052 ... -0.27615266 -0.5340997
    0.56862998]
  ...
  [-0.91390051  0.85987388 -1.17711962 ... -2.26343961 -0.24527301
    0.62290058]
  [-0.42451159  0.78017126 -0.26212925 ... -0.87759747 -0.60785208
    0.89716683]
  [-0.56939613  0.92875972  0.67831698 ... -1.44741956 -2.11591425
   -0.0

/tmp/ipykernel_33641/3663964180.py:19: RuntimeWarning: divide by zero encountered in divide
  Pk = 1/np.abs(kk**n)
